<div align="left"><img width="40%" src="templates/logo_fmriflows.png"></div>

# Welcome to fMRIflows

fMRIflows is a consortium of many (dependent) fMRI analysis pipelines, including anatomical and functional pre-processing, univariate 1st and 2nd-level analysis, as well as multivariate pattern analysis.

This notebook will help you to setup the JSON specification files to run the individual analyses.

# Exploration of the dataset

First, let's see what we've got:

In [ ]:
from bids.layout import BIDSLayout
layout = BIDSLayout("/data/")

### List of subjects

In [ ]:
subject_list = layout.get_subjects()
subject_list

### Modalities in the dataset

In [ ]:
modalities = layout.get_modalities()
modalities

In [ ]:
modality_anat = layout.get_types(modality='anat')
modality_anat

In [ ]:
modality_func = layout.get_types(modality='func')
modality_func

### Sessions in the dataset

In [ ]:
session_list = layout.get_sessions()
session_list

### Runs in the dataset

In [ ]:
runs = layout.get_runs()
runs

### Tasks in the dataset

In [ ]:
task_id = layout.get_tasks()
task_id

### Metadata in dataset

To get information, such as **TR** and **voxel resolution** of functional images, let's collect the metadata from the functional images of all subjects (of the first task).

In [ ]:
# List of functional images
func_files = layout.get(modality='func', return_type='file', type='bold', task=task_id[0])

Next, let's collect TR and voxel resolution of all functional images (of first task), overall subjects, sessions and runs.

In [ ]:
import numpy as np
import nibabel as nb

resolution = np.array([nb.load(f).header.get_zooms() for f in func_files])

In [ ]:
# Get median TR of all collected functional images
TR = np.median(resolution[:, 3])
TR

In [ ]:
# Get average voxel resolution of all collected functional images
vox_res = [round(r, 3) for r in np.mean(resolution[:, :3], axis=0)]
vox_res

# Specifications for Preprocessing workflows

Now that we know the content of our dataset, let's write the specification file for the anatomical and functional preprocessing workflow.

## Specification for the anatomical preprocessing workflow

For the anatomical preprocessing workflow, we need only a few parameters:

In [ ]:
# Create an empty dictionary
content_anat = {}

In [ ]:
# List of subject identifiers
content_anat['subject_list_anat'] = subject_list

In [ ]:
# List of session identifiers
content_anat['session_list_anat'] = session_list

In [ ]:
# T1w image identifier (default: T1w)
content_anat['T1w_id'] = 'T1w'

In [ ]:
# Voxel resolution of reference template 
content_anat['res_norm'] = [1.0, 1.0, 1.0]

In [ ]:
# Should ANTs Normalization a 'fast' or a 'precise' normalization (default: 'precise')
content_anat['norm_accuracy'] = 'precise'

To make sure that everything is as we want it, let's plot the parameters for the anatomical preprocessing pipeline again:

In [ ]:
content_anat

## Specification for the functional preprocessing workflow

For the functional preprocessing workflow, we need a few more parameters:

In [ ]:
# Create an empty dictionary
content_func = {}

In [ ]:
# List of subject identifiers
content_func['subject_list_func'] = subject_list

In [ ]:
# List of session identifiers
content_func['session_list_func'] = session_list

In [ ]:
# List of task identifiers
content_func['task_list'] = task_id

In [ ]:
# List of run identifiers
content_func['run_list'] = runs

In [ ]:
# Reference timepoint for slice time correction (in ms)
content_func['ref_timepoint'] = int(round((TR * 1000.) / 2.0, 3))

In [ ]:
# Isometric voxel resolution for the normalization of the functional images
content_func['res_func'] = round(np.median(vox_res).astype('float64'), 3)

In [ ]:
# List of spatial filters (smoothing) to apply (separetely, i.e. with iterables)
# Values are given in mm
content_func['filters_spatial'] = [["LP", 3. * content_func['res_func']]]

In [ ]:
# List of temporal filters to apply (separetely, i.e. with iterables)
# Values are given in seconds
content_func['filters_temporal'] = [[None, 100.]]

And for the confound sub-workflow, we need to specify the **number of `CompCor` components** that should be computed, the **thresholds** to detect **outliers** in `FD`, `DVARS`, `TV`, `GM`, `WM`, `CSF`, as well as the number of **independent components** that should be extracted from the preprocessed signal.

In [ ]:
# Number of CompCor components to compute
content_func['n_compcor_confounds'] = 5

In [ ]:
# Threshold for outlier detection (3.27 represents a threshold of 99.9%)
content_func['outlier_thresholds'] = [3.27, 3.27, 3.27, 3.27, None, None]

In [ ]:
# Number of independent components to compute
content_func['n_independent_components'] = 10

To make sure that everything is as we want it, let's plot the parameters for the functional preprocessing pipeline again:

In [ ]:
content_func

## Creating the `JSON` specification file

We will be using one common `JSON` file for the specifications for the anatomical and functional preprocessing pipelines. The creation of this file is rather simple:

In [ ]:
content = {}
content.update(content_anat)
content.update(content_func)

The only thing that we're still missing is the number of parallel processes that we want to allow during the execution of the preprocessing workflows:

In [ ]:
import multiprocessing
n_proc = multiprocessing.cpu_count() - 1
n_proc

In [ ]:
content['n_parallel_jobs'] = n_proc

Now, we're ready to write the `content` to a `JSON` file. By default the filename is `fmriflows_spec_preproc.json`:

In [ ]:
import json
file_id = 'fmriflows_spec_preproc'
with open('/data/%s.json' % file_id, 'w') as f:
    json.dump(content, f, indent=4)

# Specifications for 1st-level Analysis Workflows

TEXT TEX